In [33]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
import time
import random
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

C:\Users\diego.guizanlopez\AppData\Local\Temp\ipykernel_12556\102537263.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())


In [34]:
import argparse
from enum import Enum
import sys
import time
import random
import pandas as pd
import numpy as np
import pathlib

In [35]:
def waitFinishLoad(driver,idLoader,needsDisapeared=False,by=By.ID,needsWait=True):
    """Espera a la carga de un elemento antes de continuar

    Args:
        driver (WebDriver): WebDriver
        idLoader (str): Id a buscar, puede ser XPath...
        needsDisapeared (bool, optional): Si el elemento a esperar tiene que aparecer o desaparecer como un CircularProgress. Defaults to False.
        by (str, optional): Tipo de By a buscar el Wait. Defaults to By.ID.
        needsWait (bool, optional): Si necesita esperar un extra para que no pueda saltar un Captcha. Defaults to False.
    """
    if idLoader is None:
        return
    WebDriverWait(driver, timeout=60).until(EC.presence_of_element_located((by, idLoader)))
    if needsDisapeared:
        WebDriverWait(driver, timeout=60).until(EC.invisibility_of_element((by, idLoader)))
    if needsWait:
        time.sleep(random.choice([0.05,0.1,0.2]))

def elementBy(query,driver,**kargs):
    """Devuelve un element|None dependiendo de la consulta

    Args:
        query (str): Busqueda a realizar del elemento
        driver (WebDriver): WebDriver

    Returns:
        Element|None: Elemento de la búsqueda
    """
    if not kargs.get("needsWait",False):
        waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    element=driver.find_element(kargs.get("by",By.XPATH),query)
    waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    return element

def elementsBy(query,driver,**kargs):
    """Devuelve una lista de elements dependiendo de la consulta

    Args:
        query (str): Busqueda a realizar del elemento
        driver (WebDriver): WebDriver

    Returns:
        List<Element>: Lista de elementos
    """
    if not kargs.get("needsWait",False):
        waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    element=driver.find_elements(kargs.get("byq",By.XPATH),query)
    waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    return element


def get_data(driver,nombre):
    base="//div[@class='wanted_top_right']"
    wantedBy = elementBy(f"{base}/div[4]",driver,idLoader="//img",by=By.XPATH).text.replace("by ","")
    date = get_element(base,"//*[contains(text(),'Date of birth')]/../div[2]",driver)
    nacionalidad =  get_element(base,"//*[contains(text(),'Nationality')]/../div[2]",driver)
    etnia =  get_element(base,"//*[contains(text(),'Ethnic')]/../div[2]",driver)
    idiomas_hablados =  get_element(base,"//*[contains(text(),'Spoken ')]/../div[2]",driver)
    df=pd.DataFrame(columns=["Nombre","Perseguido","Fecha","Nacionalidad","Etnia","Idiomas"])
    df['Nombre']=[nombre]
    df['Perseguido']=[wantedBy]
    df['Fecha']=[date]
    df['Nacionalidad']=[nacionalidad]
    df['Etnia']=[etnia]
    df['Idiomas']=[idiomas_hablados]
    df.set_index("Nombre",inplace=True)
    return df

def get_element(base,query,driver):
    try:
        return elementBy(f"{base}{query}",driver,by=By.XPATH).text
    except:
        return np.nan

In [36]:
URL_BASE="https://eumostwanted.eu"

In [37]:
driver.get(URL_BASE)

In [38]:
buscados=elementsBy("//div[@class='wanted_list']/div",driver,needsWait=True,by=By.CLASS_NAME,idLoader="wanted_list")
df=pd.DataFrame([])
for buscado in buscados:
    nombre=buscado.get_dom_attribute("title")
    if nombre:
        array=nombre.lower().split(",")
        if len(array)>1:
            array[1]=array[1].strip()
            array[1]=array[1].replace(" ","-")
        nombre=" ".join(array)
        if buscado.get_dom_attribute('href'):
            driver.execute_script(f"window.open('{URL_BASE}/#{buscado.get_dom_attribute('href')}')")
            driver.switch_to.window(driver.window_handles[1])
            df=pd.concat([df,get_data(driver,nombre)])
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
        
df

                Perseguido                         Fecha Nacionalidad  \
Nombre                                                                  
garweg burkhard    Germany  September 1, 1968 (55 years)       German   

                    Etnia Idiomas  
Nombre                             
garweg burkhard  European  German  
                   Perseguido                         Fecha Nacionalidad  \
Nombre                                                                     
garweg burkhard       Germany  September 1, 1968 (55 years)       German   
staub ernst-volker    Germany   October 30, 1954 (69 years)       German   

                       Etnia Idiomas  
Nombre                                
garweg burkhard     European  German  
staub ernst-volker  European  German  
                   Perseguido                         Fecha Nacionalidad  \
Nombre                                                                     
garweg burkhard       Germany  September 1, 1968 (55 years)

,Perseguido,Fecha,Nacionalidad,Etnia,Idiomas
Nombre,,,,,
garweg burkhard,Germany,"September 1, 1968 (55 years)",German,European,German
staub ernst-volker,Germany,"October 30, 1954 (69 years)",German,European,German
jankevics māris,Latvia,"January 24, 1979 (45 years)",Latvian,European,NaN
toloun farid,France,"December 27, 1972 (51 years)",French,African,Arabic French
bagaméri tamás,,"November 8, 1990 (33 years)",Hungarian,European,Hungarian
omoregie felix,Belgium,"October 22, 1977 (46 years)",Nigerian,African,NaN
leijdekkers joseph-johannes-(jos),,"July 1, 1991 (32 years)",Dutch,European,NaN
kurkuchev nikolay-shterev,Spain,"July 22, 1968 (55 years)",Bulgarian,European,Bulgarian
zámečníková eva,,"October 16, 1981 (42 years)",Slovak,NaN,Slovak


In [ ]:
path=str(pathlib.Path().resolve())+"/"
df.to_parquet(path+"datos.parquet")

In [41]:
df.groupby("Perseguido")['Perseguido'].value_counts()

Perseguido
                  28
Austria            1
Belgium            1
Bulgaria           2
Croatia            1
Czech Republic     2
Denmark            1
France             3
Germany            2
Hungary            1
Italy              1
Latvia             2
Luxembourg         2
Malta              1
Poland             1
Romania            2
Slovakia           2
Spain              2
Sweden             1
Name: count, dtype: int64